In [1]:
from ortools.linear_solver import pywraplp
from ortools.linear_solver import pywraplp
import numpy as np
from itertools import combinations
import pandas as pd
import numpy as np

In [2]:
dose_matrix = pd.read_csv('DoseMatrix.csv')

In [3]:
dose_matrix =np.array(dose_matrix)

In [4]:
CTV_index=[167, 168, 169, 170, 171, 172, 173, 186, 187, 188, 189, 190, 
     191, 192, 193, 194, 206, 207, 208, 209, 210, 211, 212, 213, 
     214, 227, 228, 229, 230, 231, 232, 233]

bladder_index=[87, 88, 89, 90, 91, 92, 93, 106, 107, 108, 109, 110, 
         111, 112, 113, 114, 126, 127, 128, 129, 130, 131, 132, 
         133, 134, 147, 148, 149, 150, 151, 152, 153]

Rectal_index=[248, 249, 250, 251, 252, 267, 268, 269, 270, 271, 272, 
        273, 287, 288, 289, 290, 291, 292, 293, 308, 309, 310, 311, 312]

Un_index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
             21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
             41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 
             61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 
             81, 82, 83, 84, 85, 86, 94, 95, 96, 97, 98, 99, 100, 101, 104, 105, 115, 118, 119, 120,
             125, 139, 140, 145, 146, 154, 159, 160, 165, 166, 174, 179, 180, 185, 199, 200, 205, 219, 220, 
             225, 226, 234, 239, 240, 241, 244, 245, 246, 247, 253, 254, 255, 258, 259, 260, 261, 262, 263, 
             264, 265, 266, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 294, 295, 296, 
             297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 313, 314, 315, 316, 317, 318, 319, 320, 
             321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 
             340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 
             359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 
             378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 
             397, 398, 399]

Left_index=[116, 117, 135, 136, 137, 138, 155, 156, 157, 158, 175, 176, 177, 178,
       195, 196, 197, 198, 215, 216, 217, 218, 235, 236, 237, 238, 256, 257]

Right_index=[101, 102, 120, 121, 122, 123, 140, 141, 142, 143, 160, 161, 162, 
       163, 180, 181, 182, 183, 200, 201, 202, 203, 220, 221, 222, 223, 241, 242]

In [5]:
CTV_index = list(map(lambda x: x-21, CTV_index))
bladder_index = list(map(lambda x: x-21, bladder_index))
Rectal_index = list(map(lambda x: x-21, Rectal_index))
Left_index = list(map(lambda x: x-21, Left_index))
Right_index = list(map(lambda x: x-21, Right_index))
Un_index = []
for i in range(400):
    if i not in CTV_index+bladder_index+Rectal_index+Left_index+Right_index:
        Un_index.append(i)

In [6]:
solver = pywraplp.Solver('Final Project',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
objective=solver.Objective() 
objective.SetMinimization()

# Decision Variables
beamlets = []
for i in range(60):
    beamlets.append(solver.NumVar(0,solver.infinity(), 'beamlets[%d]' % i))
voxels = []    
for i in range(400):
    voxels.append(solver.NumVar(0,solver.infinity(),'voxels[%d]' % i))

    
#Constraints
voxel_constraints = [0 for _ in range(400)]
for i in range(400):
    voxel_constraints[i] = solver.Constraint(0,0)
    for j in range(60):
        voxel_constraints[i].SetCoefficient(beamlets[j],dose_matrix[i,j+1])
    voxel_constraints[i].SetCoefficient(voxels[i], -1)
        
######################################################## CTV#################################################

# every voxel receives a uniform dose of 82.8       ( = 82.8)
z_p = [0 for _ in range(len(CTV_index))]
z_n = [0 for _ in range(len(CTV_index))]

for i in range(len(CTV_index)):
    z_p[i] = solver.NumVar(0, 82.8*0.08, "Z plus CTV %d" %i)
    z_n[i] = solver.NumVar(0, 82.8*0.05, "Z minus CTV %d" %i)
     
CTV_constraint = [0 for _ in range(len(CTV_index))]
for i in range(len(CTV_index)):
    CTV_constraint[i] = solver.Constraint(82.8,82.8)
    
    for j in range(60):
        CTV_constraint[i].SetCoefficient(beamlets[j],dose_matrix[CTV_index[i],j+1])
    CTV_constraint[i].SetCoefficient(z_p[i],1)
    CTV_constraint[i].SetCoefficient(z_n[i],-1)

    
    
############################################ bladder ####################################################

# # max dose to a voxel 81 ( < = 81)
z_b_1 = [0 for _ in range(len(bladder_index))]

for i in range(len(bladder_index)):
    z_b_1[i] = solver.NumVar(0, solver.infinity(), "z bladder #1 %d" %i)
    

bladder_constraint_1 = [0 for _ in range(len(bladder_index))]
for i in range(len(bladder_index)):
    bladder_constraint_1[i] = solver.Constraint(-solver.infinity(),81)
    for j in range(60):
        bladder_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[bladder_index[i],j+1])
    bladder_constraint_1[i].SetCoefficient(z_b_1[i],-1)


# Average dose should be <= 50     ( <= 50)

z_b_2 = solver.NumVar(0, solver.infinity(), "z bladder #2 %d" %i)
bladder_constraint_2 = solver.Constraint(-solver.infinity(), 50)
bladder_constraint_2.SetCoefficient(z_b_2, -1)

for i in bladder_index:
    bladder_constraint_2.SetCoefficient(voxels[i],1/len(bladder_index))

        
    
# at most 10% of the bladder should receive a dose >65          ( >65)
y = [0 for _ in range(len(bladder_index))]
z_y = solver.IntVar(0,1, "z y constraint bladder")

for i in range(len(bladder_index)):
     y[i] = solver.IntVar(0,1,"y bladder %d" %i)

bladder_constraint_3 = [0 for _ in range(len(bladder_index))]
for i in range(len(bladder_index)):
    bladder_constraint_3[i] = solver.Constraint(-solver.infinity(),65)
    for j in range(60):
        bladder_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[bladder_index[i],j+1])
    bladder_constraint_3[i].SetCoefficient(y[i],-100)

bladder_constraint_3b = solver.Constraint(-solver.infinity(),0.1*len(bladder_index))
for j in range(len(bladder_index)):
     bladder_constraint_3b.SetCoefficient(y[j],1)
bladder_constraint_3b.SetCoefficient(z_y, -1)

        
        

        
# ####################### Rectal########################################

#max dose to a voxel: 79.2              ( < = 79.2)

z_r_1 = [0 for _ in range(len(Rectal_index))]

for i in range(len(Rectal_index)):
    z_r_1[i] = solver.NumVar(0, solver.infinity(), "z rectal #1 %d" %i)
    

Rectal_constraint_1 = [0 for _ in range(len(Rectal_index))]
for i in range(len(Rectal_index)):
    Rectal_constraint_1[i] = solver.Constraint(-solver.infinity(),79.2)
    for j in range(60):
        Rectal_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Rectal_index[i],j+1])
    Rectal_constraint_1[i].SetCoefficient(z_r_1[i],-1)

    
    
# #average dose should be <= 40

z_r_2 = solver.NumVar(0, solver.infinity(), "z rectal #2 %d" %i)
Rectal_constraint_2 = solver.Constraint(-solver.infinity(), 40)
Rectal_constraint_2.SetCoefficient(z_r_2, -1.0)

for i in Rectal_index:
    Rectal_constraint_2.SetCoefficient(voxels[i],1/len(Rectal_index))



# ############################## Unspecified ############################

# max dose to a voxel: 72         ( < =72)

z_u_1 = [0 for _ in range(len(Un_index))]

for i in range(len(Un_index)):
    z_u_1[i] = solver.NumVar(0, solver.infinity(), "z Unspecified #1 %d" %i)
    

Un_constraint_1 = [0 for _ in range(len(Un_index))]
for i in range(len(Un_index)):
    Un_constraint_1[i] = solver.Constraint(-solver.infinity(),72)
    for j in range(60):
        Un_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Un_index[i],j+1])
    Un_constraint_1[i].SetCoefficient(z_u_1[i],-1)


#############################Left femur head ##########################

#max dose to a voxel: 50            ( <= 50)
  
z_l_1 = [0 for _ in range(len(Left_index))]

for i in range(len(Left_index)):
    z_l_1[i] = solver.NumVar(0, solver.infinity(), "z left femur #1 %d" %i)
    

left_constraint_1 = [0 for _ in range(len(Left_index))]
for i in range(len(Left_index)):
    left_constraint_1[i] = solver.Constraint(-solver.infinity(),50)
    for j in range(60):
        left_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Left_index[i],j+1])
    left_constraint_1[i].SetCoefficient(z_l_1[i],-1)    
        

        
# at most 10% of the left femur should receive a dose > 40 
y_left = [0 for _ in range(len(Left_index))]
z_y_left = solver.IntVar(0,1, "z y constraint left femur")

for i in range(len(Left_index)):
     y_left[i] = solver.IntVar(0,1,"y left femur %d" %i)

Left_constraint_3 = [0 for _ in range(len(Left_index))]
for i in range(len(Left_index)):
    Left_constraint_3[i] = solver.Constraint(-solver.infinity(),40)
    for j in range(60):
        Left_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[Left_index[i],j+1])
    Left_constraint_3[i].SetCoefficient(y_left[i],-100)

Left_constraint_3b = solver.Constraint(-solver.infinity(),0.15*len(Left_index))
for j in range(len(Left_index)):
     Left_constraint_3b.SetCoefficient(y_left[j],1)
Left_constraint_3b.SetCoefficient(z_y_left,-1)


############################# right femur head ##########################


#max dose to a voxel: 50               ( <= 50 )
z_rl_1 = [0 for _ in range(len(Right_index))]

for i in range(len(Right_index)):
    z_rl_1[i] = solver.NumVar(0, solver.infinity(), "z right femur #1 %d" %i)
    

Right_constraint_1 = [0 for _ in range(len(Right_index))]
for i in range(len(Right_index)):
    Right_constraint_1[i] = solver.Constraint(-solver.infinity(),50)
    for j in range(60):
        Right_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Right_index[i],j+1])
    Right_constraint_1[i].SetCoefficient(z_rl_1[i],-1)    
        

        
# at most 10% of the right femur should receive a dose > 40 
y_right = [0 for _ in range(len(Right_index))]
z_y_right = solver.IntVar(0,2, "z y constraint right femur")

for i in range(len(Right_index)):
     y_right[i] = solver.IntVar(0,1,"y right femur %d" %i)

Right_constraint_3 = [0 for _ in range(len(Right_index))]
for i in range(len(Right_index)):
    
    # <= 40
    Right_constraint_3[i] = solver.Constraint(-solver.infinity(),40)
    for j in range(60):
        Right_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[Right_index[i],j+1])
    # Big M constraint
    Right_constraint_3[i].SetCoefficient(y_right[i],-100)

Right_constraint_3b = solver.Constraint(-solver.infinity(),0.15*len(Right_index))
for j in range(len(Right_index)):
     Right_constraint_3b.SetCoefficient(y_right[j],1)
Right_constraint_3b.SetCoefficient(z_y_right, -1)
        
        
########################## Objective Function ###########################

# CTV
for i in range(len(z_p)):
    objective.SetCoefficient(z_p[i],1000)
    objective.SetCoefficient(z_n[i],1000)

# Bladders
for i in range(len(z_b_1)):
    objective.SetCoefficient(z_b_1[i],1)

objective.SetCoefficient(z_b_2,100)

# Rectal Solids
for i in range(len(z_r_1)):
    objective.SetCoefficient(z_r_1[i],1)
    
objective.SetCoefficient(z_r_2,6000)

#Unspecified
for i in range(len(z_u_1)):
    objective.SetCoefficient(z_u_1[i],1)

# left femur head
for i in range(len(z_l_1)):
    objective.SetCoefficient(z_l_1[i],5000)

#right femur head
for i in range(len(z_rl_1)):
    objective.SetCoefficient(z_rl_1[i],1)

# bladder, left femur, right femur's at most constraint
objective.SetCoefficient(z_y,1)
objective.SetCoefficient(z_y_left,1)
objective.SetCoefficient(z_y_right,1)

status = solver.Solve()

if status == solver.OPTIMAL:
    print('Problem solved in %f milliseconds' %solver.wall_time())
elif status == solver.FEASIBLE:
    print('Solver claims feasibility but not optimality')
else:
    print('Solver ran to completion but did not find an optimal solution')

print('error',objective.Value()) 


#print(bladder_constraint_2.dual_value())

#print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

Problem solved in 12400.000000 milliseconds
error 303650.98651829123


In [7]:
print(bladder_constraint_2.dual_value())

0.0


In [8]:
#print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

## All Voxels

In [9]:
All_voxels = np.zeros((400)).astype(np.float)
for i in range(400):
    All_voxels[i] = voxels[i].solution_value()
#print(All_voxels.reshape(20,20))
All_voxels = pd.DataFrame(All_voxels.reshape((20,20)), index=[i for i in range(1,21)],
                          columns = [i for i in range(1,21)]).round(2)
All_voxels

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,79.66,63.93,51.48,44.19,38.93,31.45,22.01,16.07,16.48,18.41,17.83,18.13,20.84,22.00,20.43,18.47,17.42,18.04,21.94,29.67
2,84.74,72.81,59.23,49.51,44.52,39.42,31.06,23.42,20.81,20.72,20.30,22.34,26.68,27.89,24.71,20.91,19.66,22.43,29.07,36.53
3,82.09,77.30,67.09,56.30,49.20,44.53,38.53,32.13,28.54,26.92,26.27,29.08,33.42,32.97,27.58,23.11,23.61,28.81,35.08,37.40
4,71.89,74.84,71.07,63.28,55.18,48.62,43.29,39.84,38.80,37.95,36.77,38.07,39.89,36.65,30.02,26.77,29.32,34.03,35.45,30.73
5,55.76,65.56,68.82,66.90,61.54,54.13,47.57,46.06,49.53,52.08,50.65,48.49,46.06,40.32,33.93,32.06,33.84,33.92,28.87,19.97
6,36.97,50.89,60.42,65.04,65.33,60.96,54.62,53.36,59.34,65.00,64.07,59.36,53.81,46.66,40.00,36.38,33.37,27.39,18.65,10.21
7,20.22,33.82,47.24,58.01,65.17,67.30,65.00,64.06,68.30,72.97,72.81,69.63,65.00,56.91,46.52,36.66,27.22,17.70,9.51,4.10
8,8.93,18.73,32.28,47.66,62.23,72.39,76.18,76.31,76.30,76.18,76.18,78.08,77.90,68.51,51.17,33.22,19.08,9.48,3.92,1.31
9,3.31,9.04,20.24,37.74,58.73,76.18,84.55,85.06,81.84,77.93,77.64,83.39,86.79,76.18,53.39,30.19,14.13,5.50,1.76,0.45
10,1.72,5.72,15.35,33.16,56.93,77.52,86.94,86.94,83.76,80.57,80.17,84.67,86.94,76.18,53.96,31.37,15.44,6.50,2.30,0.67


In [10]:
OUTPUT  = 'output/cross_section_position.csv'
cross_section = All_voxels.to_csv(OUTPUT, index=[i for i in range(1,21)], columns = [i for i in range(1,21)])

## Check Constraint

**Beamlets Check**

In [11]:
# check beamlets value
beamlets_value = []
for i in range(60):
    beamlets_value.append(beamlets[i].solution_value())
print('Beamlets values are ')
print(np.array(beamlets_value))   

Beamlets values are 
[20.55970392  0.          0.          0.          0.          0.
  0.         36.52299544 13.1127899   0.          0.          0.
  0.          0.          0.          0.          0.         27.9853068
 18.60689836  0.          0.          0.          0.         32.9550896
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.         15.59957222
  0.          0.          0.          0.          0.          0.
 25.50722382  0.          9.47975179 56.46998712  0.         53.5601302
  0.          0.          0.         28.24880935 35.71354163 24.06832245
  0.         45.58915876  0.          0.          0.          0.        ]


**CTV check**

In [12]:
#Every voxel receives a uniform dose of 82.8 
CTV_voxel = []
every = 0
for i in CTV_index:
    every = voxels[i].solution_value() 
    CTV_voxel.append(every)
print('CTV voxel dose(Gy)')
print(np.array(CTV_voxel))

CTV voxel dose(Gy)
[76.176      76.30531598 76.29912281 76.176      76.176      78.08153973
 77.90098743 76.176      84.54652825 85.06297181 81.83948787 77.9284102
 77.63883615 83.39126232 86.79116593 76.176      77.51876689 86.94
 86.94       83.75862756 80.5735029  80.1682479  84.6690191  86.94
 76.176      82.18225277 82.59258355 82.65064755 82.8        81.94123001
 81.78313229 79.73935537]


In [13]:
# the distance to 82.8
zp0 = np.array([[x.solution_value() for x in z_p]])
zn0 = np.array([[x.solution_value() for x in z_n]])
print('Positive distance to 82.8 \n',zp0)
print('\n')
print('Negative distance to 82.8 \n',zn0)

Positive distance to 82.8 
 [[6.624      6.49468402 6.50087719 6.624      6.624      4.71846027
  4.89901257 6.624      0.         0.         0.96051213 4.8715898
  5.16116385 0.         0.         6.624      5.28123311 0.
  0.         0.         2.2264971  2.6317521  0.         0.
  6.624      0.61774723 0.20741645 0.14935245 0.         0.85876999
  1.01686771 3.06064463]]


Negative distance to 82.8 
 [[0.         0.         0.         0.         0.         0.
  0.         0.         1.74652825 2.26297181 0.         0.
  0.         0.59126232 3.99116593 0.         0.         4.14
  4.14       0.95862756 0.         0.         1.8690191  4.14
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]


In [14]:
print('maximum difference of CTV voxels values to 82.8 are ', (max(CTV_voxel)-82.8)/82.8, (82.8- min(CTV_voxel))/82.8)

maximum difference of CTV voxels values to 82.8 are  0.05000000000000893 0.08000000000003135


In [15]:
print('maximum CTV voxels value is ', max(CTV_voxel))

maximum CTV voxels value is  86.94000000000074


In [16]:
print('minimum CTV voxels value is ', min(CTV_voxel))

minimum CTV voxels value is  76.1759999999974


In [17]:
print('percentage of voxels below 79:', sum((np.array(CTV_voxel) < 79)*1) / len(CTV_voxel))

percentage of voxels below 79: 0.40625


In [18]:
print('percentage of voxels Above 86.94:', sum((np.array(CTV_voxel) > 82.8*1.05)*1) / len(CTV_voxel))

percentage of voxels Above 86.94: 0.03125


**Bladder Check**

In [19]:
# Max dose to a voxel: 81
bladder_voxel = []
every = 0
for i in bladder_index:
    every = round(voxels[i].solution_value(),4)
    bladder_voxel.append(every)
print('Bladder Voxel Doses\n',np.array(bladder_voxel))

Bladder Voxel Doses
 [43.2862 39.8437 38.7974 37.9507 36.7672 38.0749 39.8875 54.1347 47.5704
 46.0627 49.5281 52.0841 50.6462 48.4924 46.0552 40.321  60.9597 54.6208
 53.3589 59.3382 65.     64.0699 59.3555 53.8123 46.6637 65.     64.0648
 68.2992 72.9705 72.8126 69.6349 65.    ]


In [20]:
# if any voxel has more than 81
zb1 = np.array([[x.solution_value() for x in z_b_1]])
print('distances to 81 if voxel value is above 81\n',zb1)

distances to 81 if voxel value is above 81
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]


In [21]:
# check if there are voxel with value more than 81
print('are there values more than 81?')
print(max(bladder_voxel) > 81)

print('how many percentage of voxel values are more than 81?')

print(round(sum((np.array(bladder_voxel) >81)*1)/len(bladder_voxel),2)*100,'%')

are there values more than 81?
False
how many percentage of voxel values are more than 81?
0.0 %


In [22]:
# average dose shouldbe <= 50
print('average dose received by bladder voxels are ',sum(bladder_voxel)/len(bladder_voxel))
print('slack variable to bring down the value to below 50: ', z_b_2.solution_value())

average dose received by bladder voxels are  53.26448125
slack variable to bring down the value to below 50:  3.26447889318345


In [23]:
# at most 10% of the bladder should receive a dose > 65
y_value = np.array([t.solution_value() for t in y])
print('voxel that are larger that 65 are indicated by 1 \n',y_value)

voxel that are larger that 65 are indicated by 1 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 1. 1. 0.]


In [24]:
test = np.array(list(map(lambda x: 1 if x > 65.00 else 0,bladder_voxel)))
print(round(sum(test)/len(test),4), ' of the voxels are above 65')

0.125  of the voxels are above 65


In [25]:
print('voxel that are larger that 65 are indicated by 1 \n',test)

voxel that are larger that 65 are indicated by 1 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0]


In [26]:
y_value == test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [27]:
print('the number of constraints we have to set loose to be above 65',z_y.solution_value())

the number of constraints we have to set loose to be above 65 1.0


**Rectal Solid Check**

In [28]:
# max dose to a voxel: 79.2
Rectal_voxel = []
every = 0
for i in Rectal_index:
    every = round(voxels[i].solution_value(),8)
    Rectal_voxel.append(every)
print('rectal voxel dose values')
print(np.array(Rectal_voxel))

rectal voxel dose values
[74.97703129 78.76234331 80.74631613 78.69528504 75.59220848 64.40162376
 66.29956081 70.26933321 71.44049447 69.21690546 67.89260056 65.5151556
 59.09950183 56.59713444 56.06194993 55.72983923 56.28717524 60.51321308
 63.04564436 45.09996447 38.93107193 38.19759749 43.3739319  54.07429969]


In [29]:
zr1 = np.array([[x.solution_value() for x in z_r_1]])
print('distances to 79.2 if voxel value is above 79.2\n',zr1)

distances to 79.2 if voxel value is above 79.2
 [[0.         0.         1.54631613 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]]


In [30]:
# check if there's value larger than 79.2
print('are there values more than 79.2?')
print(max(Rectal_voxel) > 79.2)

print('how many percentage of voxel values are more than 79.2?')

print(round(sum((np.array(Rectal_voxel) >79.2)*1)/len(Rectal_voxel),2)*100,'%')

are there values more than 79.2?
True
how many percentage of voxel values are more than 79.2?
4.0 %


In [31]:
# average dose should be <= 40
print('average dose received by bladder voxels are ',sum(Rectal_voxel)/len(Rectal_voxel))
print('slack variable to bring down the value to below 40: ',z_r_2.solution_value())

average dose received by bladder voxels are  62.11750757125
slack variable to bring down the value to below 40:  22.117507570425275


**Unspecified**

In [32]:
# Max dose to a voxel: 72
Un_voxel = []
every = 0
for i in Un_index:
    every = round(voxels[i].solution_value(),3)
    Un_voxel.append(every)
np.array(Un_voxel)

array([79.662, 63.928, 51.484, 44.191, 38.928, 31.449, 22.011, 16.071,
       16.483, 18.405, 17.828, 18.133, 20.84 , 21.995, 20.432, 18.471,
       17.416, 18.037, 21.937, 29.673, 84.737, 72.81 , 59.233, 49.514,
       44.524, 39.422, 31.063, 23.418, 20.812, 20.715, 20.298, 22.345,
       26.68 , 27.886, 24.715, 20.914, 19.664, 22.434, 29.07 , 36.53 ,
       82.085, 77.301, 67.086, 56.297, 49.2  , 44.532, 38.532, 32.135,
       28.543, 26.922, 26.273, 29.079, 33.425, 32.97 , 27.579, 23.11 ,
       23.612, 28.808, 35.077, 37.395, 71.892, 74.841, 71.068, 63.28 ,
       55.178, 48.619, 36.651, 30.016, 26.769, 29.318, 34.025, 35.449,
       30.725, 68.815, 66.899, 61.541, 33.931, 33.915, 28.868, 65.044,
       65.329, 18.648, 58.01 , 65.175, 67.3  , 56.915,  9.511, 47.662,
       62.227, 72.391, 68.513,  3.922, 37.741, 58.729,  1.759, 33.156,
       56.925,  2.3  , 36.498, 57.826, 75.083, 69.696,  5.716,  2.171,
       30.16 , 45.411, 60.083, 69.648, 72.958, 71.125, 61.761, 49.922,
      

In [33]:
print('Are there voxel values more than 72?')
max(Un_voxel) > 72

Are there voxel values more than 72?


True

In [34]:
print('how many percentage of voxel values are more than 72?')

print(round(sum((np.array(Un_voxel) >72)*1)/len(Un_voxel),2)*100,'%')

how many percentage of voxel values are more than 72?
4.0 %


**Left Femur Head**

In [35]:
# max dose to be voxel 50
Left_voxel = []
every = 0
for i in Left_index:
    every = round(voxels[i].solution_value(),2)
    Left_voxel.append(every)
print('left femur head voxel values')
np.array(Left_voxel)

left femur head voxel values


array([32.06, 33.84, 40.  , 36.38, 33.37, 27.39, 46.52, 36.66, 27.22,
       17.7 , 51.17, 33.22, 19.08,  9.48, 53.39, 30.19, 14.13,  5.5 ,
       53.96, 31.37, 15.44,  6.5 , 52.94, 36.07, 22.43, 12.33, 40.  ,
       31.27])

In [36]:
zl1 = np.array([[x.solution_value() for x in z_l_1]])
print('distances to 79.2 if voxel value is above 50\n',zl1)

distances to 79.2 if voxel value is above 50
 [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         1.17017694 0.
  0.         0.         3.38680897 0.         0.         0.
  3.95957661 0.         0.         0.         2.9430803  0.
  0.         0.         0.         0.        ]]


In [37]:
print('are there values more than 50?')
print(max(Left_voxel) > 50)

print('how many percentage of voxel values are more than 50?')

print(round(sum((np.array(Left_voxel) >50)*1)/len(Left_voxel),2)*100,'%')

are there values more than 50?
True
how many percentage of voxel values are more than 50?
14.000000000000002 %


In [38]:
# at most 15% of the left femur head should receive > 40
y_left_value = np.array([t.solution_value() for t in y_left])
print('voxel that are larger that 40 are indicated by 1 \n',y_left_value)

voxel that are larger that 40 are indicated by 1 
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0.]


In [39]:
test_left_femur = np.array(list(map(lambda x: 1 if x >40.00 else 0,Left_voxel)))
print(round(sum(test_left_femur)/len(test_left_femur),4), ' of the left femur voxels are above 40')

0.1786  of the left femur voxels are above 40


In [40]:
print('voxel that are larger that 40 are indicated by 1 \n',test_left_femur)

voxel that are larger that 40 are indicated by 1 
 [0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0]


In [41]:
test_left_femur == y_left_value

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [42]:
print('the number of constraints we have to set loose to be above 40',z_y_left.solution_value())

the number of constraints we have to set loose to be above 40 1.0


**Right Femur Head**

In [43]:
# max dose to a voxel: 50 
Right_voxel = []
every = 0
for i in Right_index:
    every = round(voxels[i].solution_value(),2)
    Right_voxel.append(every)
print("right femur head voxel values")
np.array(Right_voxel)

right femur head voxel values


array([55.76, 65.56, 19.97, 36.97, 50.89, 60.42, 10.21, 20.22, 33.82,
       47.24,  4.1 ,  8.93, 18.73, 32.28,  1.31,  3.31,  9.04, 20.24,
        0.45,  1.72,  5.72, 15.35,  0.67,  3.18,  8.53, 19.19,  8.36,
       17.35])

In [44]:
# slack variable needed to fullfill the constraints
zrl1 = np.array([[x.solution_value() for x in z_rl_1]])
print('distances to 79.2 if voxel value is above 50\n',zrl1)

distances to 79.2 if voxel value is above 50
 [[ 5.762998   15.55636267  0.          0.          0.8860241  10.41823422
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]]


In [45]:
# check if there's any right voxel value that is bigger than 50
print('are there values more than 50?')
print(max(Right_voxel) > 50)

print('how many percentage of voxel values are more than 50?')

print(round(sum((np.array(Right_voxel) >50)*1)/len(Right_voxel),2)*100,'%')

are there values more than 50?
True
how many percentage of voxel values are more than 50?
14.000000000000002 %


In [46]:
# at most 15% of the right  femur head should receive > 40
y_right_value = np.array([t.solution_value() for t in y_right])
print('voxel that are larger that 40 are indicated by 1 \n',y_right_value)

voxel that are larger that 40 are indicated by 1 
 [1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [47]:
test_right = np.array(list(map(lambda x: 1 if x > 40.00 else 0,Right_voxel)))
print(round(sum(test_right)/len(test_right),4), ' of the right femur voxels are above 40')

0.1786  of the right femur voxels are above 40


In [48]:
print('voxel that are larger that 40 are indicated by 1 \n',test_right)

voxel that are larger that 40 are indicated by 1 
 [1 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [49]:
test_right == y_right_value

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [50]:
print('the number of constraints we have to set loose to be above 40',z_y_right.solution_value())

the number of constraints we have to set loose to be above 40 1.0
